In [32]:
# Imports

import math
from itertools import product
import pandas as pd
import numpy as np
import random

In [33]:
def valid_emissions(seq_em, s_emission):
  for ch in seq_em:
    if ch not in s_emission:
      return False
  return True

In [34]:
def valid_transition(seq, m_tr, s_states, v_initial):
  # print("validating path for", seq)
  n = len(seq)
  m = 0
  path_possible = True
  j = s_states.index(seq[0])
  if v_initial[j] <= 0:
    path_possible = False
  
  s = 0
  while m < n and s < len(seq)-1 and path_possible:
    i = s_states.index(seq[s])
    j = s_states.index(seq[s+1])
    # seq = rest(seq)
    s += 1
    # print("transmission value is",m_tr[i][j])
    if abs(m_tr[i][j]) <= 0:
      path_possible = False
    m += 1
  
  return path_possible


In [35]:
def emission_set(e, m_emission, s_emission, s_states):
  s_emission_set = []
  j = s_emission.index(e)
  for i in range(len(s_states)):
    if abs(m_emission[i][j]) > 0:
      s_emission_set.append(s_states[i])
  print("Emission set for", e, "is ", s_emission_set)
  return s_emission_set

In [36]:
def calculate_probability(seq_states, seq_emission, s_states, s_emission, m_tr, m_emission, v_initial):
  n_seq = len(seq_states)
  row_emission = s_states.index(seq_states[0])
  col_emission = s_emission.index(seq_emission[0])
  prob_path = v_initial[row_emission] * m_emission[row_emission][col_emission]
  i = 0
  while i<n_seq-1:
    j = i+1
    row_tr = s_states.index(seq_states[i])
    col_tr = s_states.index(seq_states[j])
    row_emission = col_tr
    col_emission = s_emission.index(seq_emission[j])
    prob_tr = m_tr[row_tr][col_tr]
    prob_em = m_emission[row_emission][col_emission]
    prob_path = prob_path * prob_em * prob_tr
    i += 1
  return prob_path

In [37]:
# HMM
s_cartesian = []
def most_probable_path(s_states, s_emission, m_tr, m_emission, v_initial, seq_em):
  n_states = len(s_states)
  n_emission = len(s_emission)
  get_path = True
  while(get_path):
    # seq_em = input("next emission sequence")
    if seq_em == 'exit':
      get_path = False
      return
    s_prob_em = []
    if valid_emissions(seq_em, s_emission):
      s_transistion = {}
      n_seq_em = len(seq_em)
      i = 0
      while i < n_seq_em:
        x = seq_em[i]
        s_prob_em.insert(i, emission_set(x, m_emission, s_emission, s_states))
        i += 1
      
      s_cartesian = list(product(*s_prob_em)) # cartesian product
      s_path = []
      print("All possible transistions", s_cartesian)
      for path_seq in s_cartesian:
        if valid_transition(path_seq, m_tr, s_states, v_initial):
          s_path.append(path_seq)
      
      print("Valid paths", s_path)
      max_seq_em_path = None
      max_seq_em_prob = 0

      for path in s_path:
        prob = calculate_probability(path, seq_em, s_states, s_emission, m_tr, m_emission, v_initial)
        print("probability of path", path, "is", prob)
        if max_seq_em_prob < prob:
          max_seq_em_prob = prob
          max_seq_em_path = path

      print("Most probable path is", max_seq_em_path)
      return max_seq_em_prob 

In [38]:
# Problem in slides

s_states = ['S1', 'S2', 'S3']
s_emission = ['X', 'Y', 'Z']
m_tr = [[0.0, 0.0, 1.0],
        [0.4, 0.6, 0.0],
        [0.7, 0.3, 0.0]]
m_emission = [[0.4, 0, 0.6],
              [0.2, 0.8, 0],
              [0, 0.7, 0.3]]
v_initial = [0.5, 0.5, 0]
seq_em = ['X','X','Z']

most_probable_path(s_states, s_emission, m_tr, m_emission, v_initial, seq_em)

Emission set for X is  ['S1', 'S2']
Emission set for X is  ['S1', 'S2']
Emission set for Z is  ['S1', 'S3']
All possible transistions [('S1', 'S1', 'S1'), ('S1', 'S1', 'S3'), ('S1', 'S2', 'S1'), ('S1', 'S2', 'S3'), ('S2', 'S1', 'S1'), ('S2', 'S1', 'S3'), ('S2', 'S2', 'S1'), ('S2', 'S2', 'S3')]
Valid paths [('S2', 'S1', 'S3'), ('S2', 'S2', 'S1')]
probability of path ('S2', 'S1', 'S3') is 0.004800000000000001
probability of path ('S2', 'S2', 'S1') is 0.0028800000000000006
Most probable path is ('S2', 'S1', 'S3')


0.004800000000000001

In [39]:
# Problem in Assignment

s_states = ['1', '2', '3', '4']
s_emission = ['A', 'B', 'C', 'D']
m_tr = [[0.2, 0.7, 0, 0.1],
        [0, 0, 0.7, 0.3],
        [0.2, 0.3, 0, 0.5],
        [0, 0.5, 0.5, 0]]
m_emission = [[0.6, 0.4, 0, 0],
              [0.2, 0.2, 0.6, 0],
              [0, 0, 0.2, 0.8],
              [0.2, 0.4, 0.2, 0.2]]
v_initial = [0.3, 0.4, 0.3, 0]
seq_em = ['A','B','D', 'C']

most_probable_path(s_states, s_emission, m_tr, m_emission, v_initial, seq_em)

Emission set for A is  ['1', '2', '4']
Emission set for B is  ['1', '2', '4']
Emission set for D is  ['3', '4']
Emission set for C is  ['2', '3', '4']
All possible transistions [('1', '1', '3', '2'), ('1', '1', '3', '3'), ('1', '1', '3', '4'), ('1', '1', '4', '2'), ('1', '1', '4', '3'), ('1', '1', '4', '4'), ('1', '2', '3', '2'), ('1', '2', '3', '3'), ('1', '2', '3', '4'), ('1', '2', '4', '2'), ('1', '2', '4', '3'), ('1', '2', '4', '4'), ('1', '4', '3', '2'), ('1', '4', '3', '3'), ('1', '4', '3', '4'), ('1', '4', '4', '2'), ('1', '4', '4', '3'), ('1', '4', '4', '4'), ('2', '1', '3', '2'), ('2', '1', '3', '3'), ('2', '1', '3', '4'), ('2', '1', '4', '2'), ('2', '1', '4', '3'), ('2', '1', '4', '4'), ('2', '2', '3', '2'), ('2', '2', '3', '3'), ('2', '2', '3', '4'), ('2', '2', '4', '2'), ('2', '2', '4', '3'), ('2', '2', '4', '4'), ('2', '4', '3', '2'), ('2', '4', '3', '3'), ('2', '4', '3', '4'), ('2', '4', '4', '2'), ('2', '4', '4', '3'), ('2', '4', '4', '4'), ('4', '1', '3', '2'), ('4', '1

0.002540159999999999

In [40]:
# problem in previous mid - 2018

s_states = ['1', '2', '3']
s_emission = ['X', 'Y', 'Z']
m_tr = [[0.2, 0.4, 0.4],
        [0.3, 0.0, 0.7],
        [0.4, 0.0, 0.6]]
m_emission = [[0.2, 0.4, 0.4],
              [0, 0.3, 0.7],
              [0.4, 0.1, 0.5]]
v_initial = [0.4, 0.6, 0]
seq_em = ['X','X','Z']

most_probable_path(s_states, s_emission, m_tr, m_emission, v_initial, seq_em)

Emission set for X is  ['1', '3']
Emission set for X is  ['1', '3']
Emission set for Z is  ['1', '2', '3']
All possible transistions [('1', '1', '1'), ('1', '1', '2'), ('1', '1', '3'), ('1', '3', '1'), ('1', '3', '2'), ('1', '3', '3'), ('3', '1', '1'), ('3', '1', '2'), ('3', '1', '3'), ('3', '3', '1'), ('3', '3', '2'), ('3', '3', '3')]
Valid paths [('1', '1', '1'), ('1', '1', '2'), ('1', '1', '3'), ('1', '3', '1'), ('1', '3', '3')]
probability of path ('1', '1', '1') is 0.0002560000000000001
probability of path ('1', '1', '2') is 0.0008960000000000003
probability of path ('1', '1', '3') is 0.0006400000000000003
probability of path ('1', '3', '1') is 0.0020480000000000008
probability of path ('1', '3', '3') is 0.003840000000000001
Most probable path is ('1', '3', '3')


0.003840000000000001

In [47]:
s_states = input("Enter the states").split(" ")
s_emission = input("Enter the emission states").split(" ")
m_tr = np.random.dirichlet(np.ones(len(s_states) ** 2),size=1).reshape((len(s_states), len(s_states)))
m_emission = np.random.dirichlet(np.ones(len(s_states) * len(s_emission)),size=1).reshape((len(s_states), len(s_emission)))
v_initial = np.random.dirichlet(np.ones(len(s_states)),size=1)[0]
seq_em = input("Enter the sequence to be checked").split(" ")

print(s_states, s_emission, m_tr, m_emission, v_initial, seq_em)
most_probable_path(s_states, s_emission, m_tr, m_emission, v_initial, seq_em)

Enter the states1 2 3
Enter the emission statesx y z
Enter the sequence to be checkedx x z
['1', '2', '3'] ['x', 'y', 'z'] [[0.06192086 0.05258039 0.19337979]
 [0.06680463 0.02468485 0.38854657]
 [0.06459168 0.08064924 0.06684198]] [[0.23203716 0.00399335 0.02312897]
 [0.05383208 0.17392656 0.1378629 ]
 [0.101313   0.09457178 0.1793342 ]] [0.58619073 0.11062541 0.30318386] ['x', 'x', 'z']
Emission set for x is  ['1', '2', '3']
Emission set for x is  ['1', '2', '3']
Emission set for z is  ['1', '2', '3']
All possible transistions [('1', '1', '1'), ('1', '1', '2'), ('1', '1', '3'), ('1', '2', '1'), ('1', '2', '2'), ('1', '2', '3'), ('1', '3', '1'), ('1', '3', '2'), ('1', '3', '3'), ('2', '1', '1'), ('2', '1', '2'), ('2', '1', '3'), ('2', '2', '1'), ('2', '2', '2'), ('2', '2', '3'), ('2', '3', '1'), ('2', '3', '2'), ('2', '3', '3'), ('3', '1', '1'), ('3', '1', '2'), ('3', '1', '3'), ('3', '2', '1'), ('3', '2', '2'), ('3', '2', '3'), ('3', '3', '1'), ('3', '3', '2'), ('3', '3', '3')]
Valid

6.77743262868904e-05